In [2]:
%load_ext autoreload
%autoreload 2

import checklist
import spacy
import itertools

import checklist.editor
import checklist.text_generation
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.test_suite import TestSuite
import numpy as np
import spacy
from checklist.perturb import Perturb


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
from checklist.pred_wrapper import PredictorWrapper

import sys
sys.path.append('/home/marcotcr/work/ml-tests/')
from mltests import model_wrapper
model = model_wrapper.ModelWrapper()
new_pp = PredictorWrapper.wrap_softmax(model.predict_proba)


In [4]:
editor = checklist.editor.Editor()
editor.tg

In [5]:
nlp = spacy.load('en_core_web_sm')

In [6]:
qs = []
labels = []
all_questions = set()
for x in open('/home/marcotcr/datasets/glue/glue_data/QQP/dev.tsv').readlines()[1:]:
    try:
        q1, q2, label = x.strip().split('\t')[3:]
    except:
        print(x)
        continue
    all_questions.add(q1)
    all_questions.add(q2)
    qs.append((q1, q2))
    labels.append(label)
labels = np.array(labels).astype(int)

"	What does the Quran say about homosexuality?	0



In [7]:
# [x for x in qs if 'better' in x[0]]

In [8]:
import pickle
spacy_map =  pickle.load(open('/home/marcotcr/tmp/processed_qqp.pkl', 'rb'))

In [9]:
parsed_qs = [(spacy_map[q[0]], spacy_map[q[1]]) for q in qs]

In [10]:
suite = TestSuite()

## Vocabulary

In [11]:
professions = editor.suggest('{first_name} works as {a:mask}.')[:30]
print(', '.join(professions))

journalist, historian, nurse, engineer, accountant, attorney, architect, artist, editor, actor, actress, analyst, intern, economist, entrepreneur, author, assistant, interpreter, investor, executive, organizer, escort, educator, agent, academic, activist, advisor, administrator, investigator, auditor


In [12]:
professions = editor.suggest('{first_name} works as {a:mask}.')[:30]
professions += editor.suggest('{first_name} {last_name} works as {a:mask}.')[:30]
professions = list(set(professions))
# professions

In [13]:
print(', '.join(editor.suggest('{first_name} {last_name} is a good {mask}.')[:30]))

guy, player, writer, actor, example, man, person, friend, kid, hitter, read, shooter, one, coach, dude, pick, teacher, shot, fighter, artist, student, quarterback, poet, character, reader, sport, name, bet, cook, reporter


In [14]:
other_nouns = ['player', 'person', 'friend', 'kid', 'candidate']
nouns = list(set(professions + other_nouns))

In [15]:
print(', '.join(editor.suggest('Is {first_name} {last_name} {a:mask} {noun}?', noun=nouns)[:50]))

effective, actual, important, American, active, successful, good, independent, outstanding, experienced, excellent, elite, honest, influential, real, interested, Australian, international, appropriate, inspiring, better, established, accomplished, serious, great, aspiring, true, outside, ethical, English, bad, credible, LGBT, acceptable, average, interesting, exceptional, Israeli, incompetent, unusual, official, expert, ordinary, adequate, amazing, Irish, political, amateur, accurate, career


In [16]:
adjs = ['effective', 'actual', 'American', 'active', 'honest', 'excellent', 'elite', 'acomplished', 'official', 'outstanding', 'experienced', 'independent', 'international', 'aspiring', 'average', 'good', 'amazing', 'exceptional', 'successful', 'accredited', 'English', 'real', 'bad', 'terrible', 'fake', 'unusual', 'influential', 'incompetent']

In [17]:
t = editor.template(('Is {first_name} {last_name} {a:noun}?', 'Is {first_name} {last_name} {a:adj} {noun}?'),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
test = MFT(**t, labels=0, name='Modifier: adj', capability='Vocabulary', 
          description = 'Adding an adjective makes questions non-duplicates')
# test.run(new_pp)
# test.summary(n=6)
suite.add(test)

Is person {a1, a2}?

In [18]:
print(', '.join(editor.suggest('Is John Wayne {mask}?')))
print()
print(', '.join(editor.suggest('Is John Wayne {a:mask}?')[:50]))

dead, gay, alive, right, Dead, real, Jewish, mad, wrong, correct, insane, back, crazy, related, OK, Right, immortal, evil, straight, interested, famous, racist, sane, next, President, alone, lying, relevant, okay, DEAD, bisexual, God, angry, guilty, ok, Alone, Wrong, ready, gone, innocent, King, human, Muslim, Black, assassinated, Batman, here, Real, dying, involved, homosexual, special, Famous, happy, free, murdered, Superman, president, joking, White, Back, Gay, Catholic, cool, safe, resurrected, killed, American, missing, corrupt, cursed, alright, single, Crazy, Mad, Satan, Missing, legit, shot, Evil, finished, worthy, doomed, sincere, aware, autistic, responsible, dangerous, Christian, Cool, Immortal, radioactive, remembered, Next, serious, cheating, different, popular, Legend, listening, now, illegitimate, homophobic, kidding, sick, smart, connected, trustworthy, Innocent, legal

racist, atheist, outlaw, asshole, actor, ape, American, inspiration, idiot, orphan, Christian, ancesto

In [19]:
adjs_without_overlap = ['dead', 'gay', 'Jewish', 'Christian', 'American', 'mad', 'immortal', 'evil', 'famous', 'racist', 'Muslim', 'white', 'black', 'English', 'autistic', 'Australian', 'trustworthy', 'an atheist', 'an anarchist', 'an inventor', 'Indian', 'Armenian', 'an astronaut', 'an immigrant']

In [20]:
t = editor.template((
    'Is {first_name} {last_name} {adj1}?',
    'Is {first_name} {last_name} {adj2}?',
    ),
    adj=adjs_without_overlap,
    remove_duplicates=True, 
    nsamples=1000)
test = MFT(**t, labels=0, name='different adjectives', capability = 'Vocabulary',
          description='Same first and last name, different adjectives')
# test.run(new_pp)
# test.summary(n=5)
suite.add(test)

different animals

In [21]:
print(', '.join(editor.suggest('I have a pet {mask}.')))

cat, dog, rabbit, turtle, too, spider, now, rat, name, goat, lizard, dragon, named, pig, wolf, monkey, girl, squirrel, owl, also, here, called, there, tiger, python, friend, lab, phone, mix, bear, bird, …, boy, farm, duck, snake, zoo, owner, problem, fish, carrier, Shiva, shop, myself, companion, boyfriend, kitten, store, dinosaur, animal, watch, ring, project, deer, Labrador, Shepherd, tree, killer, lobster, mom, dish, X, bunny, girlfriend, shark, already, gun, car, puppy, one, computer, elephant, fetish, bug, brother, lover, tattoo, monitor, snail, cow, XD, Goat, mouse, somewhere, robot, sometimes, mother, syndrome, haha, pair, knife, life, carriage, Persian, cats, collar, that, diary, door, family, boo, number, show, assistant, toy, horse, chicken, laptop, server, tracker


In [22]:
animals = ['cat', 'dog', 'rabbit', 'turtle', 'spider', 'rat', 'goat', 'lizard', 'pig', 'monkey', 'squirrel', 'owl', 'snake', 'fish', 'lobster', 'snail', 'chicken']

In [23]:
print(', '.join(editor.suggest('Can I feed my {an} {mask}?', an=animals)))

food, pellets, eggs, water, again, worms, properly, now, meat, today, too, anything, outside, here, something, tonight, poop, fish, this, milk, directly, that, myself, rice, instead, nuts, there, live, steak, more, dinner, formula, treats, poison, regularly, correctly, rabbit, right, blood, enough, back, chicken, cheese, once, larvae, peanuts, feed, alive, honey, grass, feces, soup, anyway, scraps, bugs, what, alone, breakfast, oil, raw, well, chocolate, urine, corn, cereal, sugar, seeds, home, liver, free, better, protein, online, daily, cookies, carrots, lunch, yogurt, candy, cat, grain, seed, spinach, anymore, salad, fat, antibiotics, greens, Rice, it, indoors, …, shit, inside, money, toys, crap, organic, naturally, hay, bacon, some, cats, bird, bamboo, egg, breast, separately, rats, juice, vegetables, biscuits, waste, butter, crow, insulin, soy, sometimes, meal, mix, bones, yet, fly, appropriately, balls, privately, bananas, tuna, salmon, chickens, Milo, birds, bread, saliva, chick

In [24]:
food = ['eggs', 'water',  'worms', 'meat', 'poop', 'milk', 'rice', 'nuts', 'steak', 'formula',  'soup', 'bugs', 'oil', 'chocolate', 'corn', 'cereal', 'sugar', 'seeds', 'liver', 'cookies', 'carrots', 'yogurt', 'salad', 'greens', 'rice', 'bananas', 'tuna', 'apples', 'salmon', 'butter', 'insulin', 'soy']

In [25]:
t = editor.template((
    'Can I feed my {animal1} {food}?',
    'Can I feed my {animal2} {food}?',
    ),
    animal=animals,
    food=food,
    remove_duplicates=True, 
    nsamples=1000)
name = 'Different animals' 
desc = 'Ask the same question about two different pet animals, expect prediction to be 0'
test = MFT(**t, labels=0, name=name, capability = 'Vocabulary',
          description=desc)
# test.run(new_pp)
# test.summary(n=5)
suite.add(test)

Modifiers that don't matter

In [26]:
action = editor.suggest('Is that {animal} really {mask} on the couch?', animal=animals)[:30]

editor.suggest('Is that {animal} {mask} {action} on the couch?', animal=animals, action=action)
non_changing_modifier = ['really', 'truly', 'actually', 'indeed', 'in fact', 'currently', 'literally', 'somehow']
t = editor.template((
    'Is that {animal} {action} on the {place}?',
    'Is that {animal} {mod2} {action} on the {place}?',
    ),
    action=action,
    animal=animals,
    mod=non_changing_modifier,
    place =['couch','bed', 'sofa', 'table'],
    remove_duplicates=False, 
    nsamples=1000)
name = 'Irrelevant modifiers - animals' 
desc = 'Add modifiers that preserve question semantics (e.g. \'really\')'
test = MFT(**t, labels=1, name=name, capability = 'Vocabulary',
          description=desc)
# test.run(new_pp)
# test.summary(n=5)
suite.add(test)

In [27]:
action = editor.suggest('Is {first_name1} {mask} to {first_name2}?')[:30]
editor.suggest('Is {first_name1} {mask} {a} to {first_name2}?', a=action)
non_changing_modifier = ['really', 'truly', 'actually', 'indeed', 'in fact']
t = editor.template((
    'Is {first_name1} {action} to {first_name2}?',
    'Is {first_name1} {mod2} {action} to {first_name2}?',
    ),
    action=action,
    mod=non_changing_modifier,
    remove_duplicates=True, 
    nsamples=1000)
name = 'Irrelevant modifiers - people' 
desc = 'Add modifiers that preserve question semantics (e.g. \'really\')'
test = MFT(**t, labels=1, name=name, capability = 'Vocabulary',
          description=desc)
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [28]:
t = editor.template((
    'My pet {animal1} eats {food}. Is it normal for animals to eat {food}?',
    'My pet {animal2} eats {food}. Is it normal for animals to eat {food}?',
    ),
    animal=animals,
    food=food,
    remove_duplicates=True, 
    nsamples=1000)
name = 'Irrelevant preamble with different examples.' 
desc = 'Ask questions about animals in general, but with different examples in the preamble. Expect duplicate.'
test = MFT(**t, labels=1, name=name, capability = 'Vocabulary',
          description=desc)
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [29]:
editor.visual_suggest('I broke my {mask} playing tennis.')

TemplateEditor(bert_suggests=['arm', 'leg', 'wrist', 'back', 'hand', 'neck', 'foot', 'ankle', 'nose', 'thumb',…

In [30]:
# nouns = editor.selected_suggestions
nouns = ['finger', 'forearm', 'feet', 'head', 'foot', 'elbow', 'nose', 'back', 'heart', 'shoulder', 'skull', 'toe', 'hip', 'neck', 'body', 'thumb', 'rib', 'knee', 'jaw', 'heel', 'thigh', 'ankle', 'arm', 'wrist', 'eye', 'spine', 'leg', 'butt', 'teeth', 'skin', 'bone', 'hand', 'face']

In [31]:
t = editor.template((
    'I hurt my {n} last time I played {sport}. {followup}',
    'I hurt my {n2} last time I played {sport}. {followup}',
    ),
    n=nouns,
    sport=['tennis', 'golf', 'soccer', 'football'],
    followup=['Is this going to impact my performance?',
              'Should I never play again?',
              'Is this a common injury?',
             'Is it normal to hurt this part of the body?',],
    remove_duplicates=True, 
    nsamples=1000)
name = 'Preamble is relevant (different injuries)' 
desc = 'Question preamble mentions different injuries, which makes the questions themselves non-duplicates.'
test = MFT(**t, labels=0, name=name, capability = 'Vocabulary',
          description=desc)
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

### Taxonomy

Synonyms

In [32]:
tmp = []
x = editor.suggest('How can I become more {mask}?')
x += editor.suggest('How can I become less {mask}?')
for a in set(x):
    e = editor.synonyms('How can I become {moreless} %s?' % a, a, moreless=['more', 'less'])
    if e:
#         print(a, [b[0][0] for b in e] )
        tmp.append([a] + e)
#         opps.append((a, e[0][0][0]))
print(',\n'.join([str(tuple(x)) for x in tmp]))

('effective', 'efficient'),
('hateful', 'mean'),
('critical', 'decisive', 'vital'),
('worried', 'upset'),
('important', 'significant', 'authoritative'),
('resilient', 'live'),
('consistent', 'uniform', 'logical', 'coherent'),
('enlightened', 'educated', 'clear'),
('corrupt', 'corrupted'),
('radical', 'revolutionary'),
('addicted', 'addict'),
('suspicious', 'suspect', 'wary'),
('insecure', 'unsafe'),
('controlling', 'moderate', 'control'),
('positive', 'confident'),
('open', 'clear', 'capable', 'receptive', 'candid'),
('professional', 'pro'),
('activist', 'militant'),
('evil', 'vicious'),
('independent', 'autonomous'),
('anxious', 'nervous'),
('charitable', 'sympathetic', 'benevolent'),
('spiritual', 'religious'),
('confident', 'positive'),
('kind', 'tolerant'),
('scared', 'frightened'),
('sensitive', 'sensible'),
('so', 'then'),
('clear', 'open', 'clean', 'light', 'clearly'),
('specific', 'particular'),
('individual', 'single', 'private', 'person'),
('passive', 'inactive', 'peaceful'),

In [33]:
synonyms = [ ('spiritual', 'religious'), ('angry', 'furious'), ('organized', 'organised'),
            ('vocal', 'outspoken'), ('grateful', 'thankful'), ('intelligent', 'smart'),
            ('humble', 'modest'), ('courageous', 'brave'), ('happy', 'joyful'), ('scared', 'frightened'),
           ]

Antonyms

In [34]:
opps = []
x = editor.suggest('How can I become more {mask}?')
x += editor.suggest('How can I become less {mask}?')
for a in set(x):
    e = editor.antonyms('How can I become {moreless} %s?' % a, a, moreless=['more', 'less'])
    if e:
#         print(a, [b[0][0] for b in e] )
        opps.append([a] + e)
#         opps.append((a, e[0][0][0]))
print(','.join([str(tuple(x)) for x in opps]))

('impatient', 'patient'),('insecure', 'secure'),('positive', 'negative', 'neutral'),('organic', 'functional'),('stupid', 'intelligent', 'smart'),('powerless', 'powerful'),('defensive', 'offensive'),('specific', 'general'),('individual', 'common'),('passive', 'active'),('dependent', 'independent'),('conservative', 'progressive', 'liberal'),('conspicuous', 'invisible'),('pessimistic', 'optimistic'),('negative', 'positive'),('progressive', 'conservative'),('cautious', 'brave'),('courageous', 'fearful'),('visible', 'invisible'),('capitalist', 'socialist'),('hungry', 'thirsty'),('bad', 'good'),('unhappy', 'happy'),('irresponsible', 'responsible'),('hopeful', 'hopeless'),('religious', 'secular'),('fat', 'lean', 'thin'),('smart', 'stupid'),('rude', 'civil', 'polite'),('emotional', 'intellectual'),('active', 'passive'),('optimistic', 'pessimistic'),('humble', 'proud'),('shy', 'confident')


In [35]:
antonyms = [('progressive', 'conservative'),('religious', 'secular'),('positive', 'negative'),('defensive', 'offensive'),('rude',  'polite'),('optimistic', 'pessimistic'),('stupid', 'smart'),('negative', 'positive'),('unhappy', 'happy'),('active', 'passive'),('impatient', 'patient'),('powerless', 'powerful'),('visible', 'invisible'),('fat', 'thin'),('bad', 'good'),('cautious', 'brave'), ('hopeful', 'hopeless'),('insecure', 'secure'),('humble', 'proud'),('passive', 'active'),('dependent', 'independent'),('pessimistic', 'optimistic'),('irresponsible', 'responsible'),('courageous', 'fearful')]

In [36]:

t = editor.template([
    (
    'How can I become more {x[0]}?',
    'How can I become more {x[1]}?',
    ),
    (
    'How can I become more {x[1]}?',
    'How can I become more {x[0]}?',
    ),
    (
    'How can I become less {x[0]}?',
    'How can I become less {x[1]}?',
    ),
    (
    'How can I become less {x[1]}?',
    'How can I become less {x[0]}?',
    ),
    (
    'How can I become {a:x[0]} person?',
    'How can I become {a:x[1]} person?',
    ),
    (
    'How can I become {a:x[1]} person?',
    'How can I become {a:x[0]} person?',
    ),
],
    unroll=True,
    x=synonyms,
    remove_duplicates=True, 
    nsamples=1000)
name = 'How can I become more {synonym}?' 
desc = 'different (simple) templates where words are replaced with their synonyms'
test = MFT(**t, labels=1, name=name, capability = 'Taxonomy',
          description=desc)
# test.run(new_pp)
# test.summary(n=3)
suite.add(test, overwrite=True)

In [37]:

import re
def replace_pairs(pairs):
    def replace_z(text):
        ret = []
        for x, y in pairs:
            t = re.sub(r'\b%s\b' % x, y, text )
            if t != text:
                ret.append(t)
            if y == 'smart':
                continue
            t = re.sub(r'\b%s\b' % y, x, text )
            if t != text:
                ret.append(t)
        return list(set(ret))
    return replace_z
def apply_and_pair(fn):
    def ret_fn(text):
        ret = fn(text)
        return [(text, r) for r in ret]
    return ret_fn

In [38]:
synonyms

[('spiritual', 'religious'),
 ('angry', 'furious'),
 ('organized', 'organised'),
 ('vocal', 'outspoken'),
 ('grateful', 'thankful'),
 ('intelligent', 'smart'),
 ('humble', 'modest'),
 ('courageous', 'brave'),
 ('happy', 'joyful'),
 ('scared', 'frightened')]

In [39]:
name = '(question, f(question)) where f(question) replaces synonyms?' 
desc = 'Expect 1, should be easy because it\'s individual word changes'
t = Perturb.perturb(list(all_questions), apply_and_pair(replace_pairs(synonyms)), nsamples=1000, keep_original=False)
test = INV(t.data, threshold=0.1, name=name, description=desc, capability='Taxonomy')
# test.run(new_pp, n=500, seed=1)
# test.summary(n=3)
suite.add(test, overwrite=True)


In [40]:
def apply_to_each_and_product(fn):
    def apply_to_one(x):
        p = fn(x)
        if not p:
            p = []
        return list(set([x] + p))
    def ret_fn(pair):
        p1 = apply_to_one(pair[0])
        p2 = apply_to_one(pair[1])
        return [x for x in itertools.product(p1, p2) if x != pair]
    return ret_fn

In [41]:
name = 'Replace synonyms in real pairs'
desc = ''
t = Perturb.perturb(qs, apply_to_each_and_product(replace_pairs(synonyms)), nsamples=1000, keep_original=True)
test = INV(t.data, threshold=0.1, name=name, description=desc, capability='Taxonomy')
# test.run(new_pp, n=500, seed=1)
# test.summary(n=3)
suite.add(test, overwrite=True)

In [42]:

t = editor.template([(
    'How can I become more {x[0]}?',
    'How can I become less {x[0]}?',
    ),
    (
    'How can I become less {x[1]}?',
    'How can I become more {x[1]}?',
    )],
    unroll=True,
    x=antonyms,
    remove_duplicates=True, 
    nsamples=1000)
name = 'How can I become more X != How can I become less X' 
desc = ''
test = MFT(**t, labels=0, name=name, capability = 'Vocabulary',
          description=desc)
# test.run(new_pp, n=500, seed=1)
# test.summary(n=3)
suite.add(test, overwrite=True)

In [43]:

t = editor.template([(
    
    'How can I become more {x[0]}?',
    'How can I become less {x[1]}?',
    ),
    (
    'How can I become less {x[0]}?',
    'How can I become more {x[1]}?',
    )],
    unroll=True,
    x=antonyms,
    remove_duplicates=True, 
    nsamples=1000)
name = 'How can I become more X = How can I become less antonym(X)' 
desc = ''
test = MFT(**t, labels=1, name=name, capability = 'Taxonomy',
          description=desc)
# test.run(new_pp, n=500, seed=1)
# test.summary(n=3)
suite.add(test, overwrite=True)

DIR version (kinda bad, won't add to suite)

In [44]:
t = Perturb.perturb(list(all_questions), apply_and_pair(replace_pairs(antonyms)), nsamples=200, keep_original=False)
test = DIR(t.data, expect=Expect.eq(0), agg_fn='all')
# test.run(new_pp)
# test.summary(n=5)
# suite.add(test)


## Robustness

In [45]:
def wrap_apply_to_each(fn, both=False, *args, **kwargs):
    def new_fn(qs, *args, **kwargs):
        q1, q2 = qs
        ret = []
        fnq1 = fn(q1, *args, **kwargs)
        fnq2 = fn(q2, *args, **kwargs)
        if type(fnq1) != list:
            fnq1 = [fnq1]
        if type(fnq2) != list:
            fnq2 = [fnq2]
        ret.extend([(x, str(q2)) for x in fnq1])
        ret.extend([(str(q1), x) for x in fnq2])
        if both:
            ret.extend([(x, x2) for x, x2 in itertools.product(fnq1, fnq2)])
        return [x for x in ret if x[0] and x[1]]
    return new_fn
def wrap_apply_to_both(fn, *args, **kwargs):
    def new_fn(qs, *args, **kwargs):
        q1, q2 = qs
        ret = []
        fnq1 = fn(q1, *args, **kwargs)
        fnq2 = fn(q2, *args, **kwargs)
        if type(fnq1) != list:
            fnq1 = [fnq1]
        if type(fnq2) != list:
            fnq2 = [fnq2]
        ret.extend([(x, x2) for x, x2 in itertools.product(fnq1, fnq2)])
        return [x for x in ret if x[0] and x[1]]
    return new_fn

Typos

In [46]:
t = Perturb.perturb(qs, wrap_apply_to_each(Perturb.add_typos), nsamples=500)
test = INV(t.data, name='add one typo', capability='Robustness', description='')
# test.run(new_pp)
# test.summary(3)
suite.add(test, overwrite=True)

Contractions

In [47]:
t = Perturb.perturb(qs, wrap_apply_to_each(Perturb.contractions, both=True), nsamples=500)
test = INV(**t, name='contrations', capability='Robustness', description='')
# test.run(new_pp)
# test.summary(3)
suite.add(test)

Paraphrases

In [48]:
import itertools
def me_to_you(text):
    t = re.sub(r'\bI\b', 'you', text)
    t = re.sub(r'\bmy\b', 'your', t)
    return re.sub(r'\bmine\b', 'yours', t)
def paraphrases(text):
    ts = ['How do I ', 'How can I ', 'What is a good way to ', 'How should I ']
    templates1 = ['How do I {x}?', 'How can I {x}?', 'What is a good way to {x}?', 'If I want to {x}, what should I do?',
                'In order to {x}, what should I do?']
    ts2 = ['Can you ', 'Can I ']#, 'Do I']
    ts3 = ['Do I ']
    templates2 = ['Can you {x}?', 'Can I {x}?', 'Do you think I can {x}?', 'Do you think you can {x}?',]
    templates3 = ['Do I {x}?', 'Do you think I {x}?']
    ret = []
    for i, (tsz, templates) in enumerate(zip([ts, ts2, ts3], [templates1, templates2, templates3])):
        for t in tsz:
            if text.startswith(t):
                x = text[len(t):].strip('?')
                ts = editor.template(templates, x=x).data[0]
                if i <= 1:
                    ts = ts + [me_to_you(x) for x in ts]
                ret += ts
    return ret
def paraphrases_product(text):
    pr = paraphrases(text)
    return list(itertools.product(pr, pr))

def paraphrase_each(pair):
    p1 = paraphrases(pair[0])
    p2 = paraphrases(pair[1])
    return list(itertools.product(p1, p2))

In [49]:
t = Perturb.perturb(list(all_questions), paraphrases_product, nsamples=200, keep_original=False)
name = '(q, paraphrase(q))'
desc = 'For questions that start with "How do I X", "How can I X", etc'
test = DIR(t.data, expect=Expect.eq(1), agg_fn='all', name=name, description=desc, capability='Robustness')
# test.run(new_pp)
# test.summary(n=5)
suite.add(test, overwrite=True)

In [50]:
t = Perturb.perturb(qs, paraphrase_each, nsamples=100, keep_original=True)
name = 'Product of paraphrases(q1) * paraphrases(q2)'
desc = 'For questions that start with "How do I X", "How can I X", etc'
test = INV(t.data, name=name, description=desc, capability='Robustness')
# test.run(new_pp)
# test.summary(n=5)
suite.add(test)


## Ner

### Change same name, number, location in both

Names

person1 and person2 are different by first and last name

In [51]:
t = editor.template((
    'Is {first_name1} {last_name1} {adj}?',
    'Is {first_name2} {last_name2} {adj}?',
    ),
    adj=adjs_without_overlap,
    remove_duplicates=True, 
    nsamples=1000)
test = MFT(**t, labels=0, name='same adjectives, different people', capability = 'NER',
          description='Different first and last name, same adjectives')
# test.run(new_pp)
# test.summary(n=5)
suite.add(test)

person1 and person2 are different by first name only

In [52]:
t = editor.template((
    'Is {first_name} {last_name} {adj}?',
    'Is {first_name2} {last_name} {adj}?',
    ),
    adj=adjs_without_overlap,
    remove_duplicates=True, 
    nsamples=1000)
test = MFT(**t, labels=0, name='same adjectives, different people v2', capability = 'NER',
          description='Different first name, same adjective and last name')
# test.run(new_pp)
# test.summary(n=5)
suite.add(test)

person1 and person2 are different by last name only

In [53]:
t = editor.template((
    'Is {first_name} {last_name} {adj}?',
    'Is {first_name} {last_name2} {adj}?',
    ),
    adj=adjs_without_overlap,
    remove_duplicates=True, 
    nsamples=1000)
test = MFT(**t, labels=0, name='same adjectives, different people v3', capability = 'NER',
          description='Different last name, same adjective and first name')
# test.run(new_pp)
# test.summary(n=5)
suite.add(test)

In [54]:
def change_both_wrapper(fn):
    def change_both(qs):
        q1, q2 = qs
        seed = np.random.randint(100)
        c1 = fn(q1, seed=seed, meta=True)
        c2 = fn(q2, seed=seed, meta=True)
        if not c1 or not c2:
            return
        c1, m1 = c1
        c2, m2 = c2
        return [(q1, q2) for q1, q2, m1, m2 in zip(c1, c2, m1, m2) if m1 == m2]
    return change_both

def change_each_wrapper(fn):
    def change_one(qs, **kwargs):
        q1, q2 = qs
        seed = np.random.randint(100)
        c1 = fn(q1, seed=seed, meta=True, **kwargs)
        c2 = fn(q2, seed=seed, meta=True, **kwargs)
        if not c1 or not c2:
            return
        c1, m1 = c1
        c2, m2 = c2
        ret = []
        ret.extend([(q1_, str(q2)) for q1_, m1_ in zip(c1, m1) if m1_[0] in str(q2)])
        ret.extend([(str(q1), q2_) for q2_, m2_ in zip(c2, m2) if m2_[0] in str(q1)])
        return ret
    return change_one
    
t = Perturb.perturb(parsed_qs, change_both_wrapper(Perturb.change_names), nsamples=500)
test = INV(**t, name='Change same name in both questions', capability='NER',
          description='')
# test.run(new_pp)
# test.summary(3)
suite.add(test)

Locs

In [55]:
t = Perturb.perturb(parsed_qs, change_both_wrapper(Perturb.change_location), nsamples=500)
test = INV(**t, name='Change same location in both questions', capability='NER',
          description='')
# test.run(new_pp)
# test.summary(3)
suite.add(test)

Numbers

In [56]:
t = Perturb.perturb(parsed_qs, change_both_wrapper(Perturb.change_number), nsamples=500)
test = INV(**t, name='Change same number in both questions', capability='NER',
          description='')
# test.run(new_pp)
# test.summary(3)
suite.add(test)

### Change name, loc, number in only one where orig prediction is duplicate

Changing only first names

In [57]:
# t = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_names), nsamples=1500, first_only=True)
t = Perturb.perturb(parsed_qs, change_each_wrapper(Perturb.change_names), nsamples=500, first_only=True)
expect_fn = Expect.eq(0)
expect_fn = Expect.slice_orig(expect_fn, lambda orig, *args: orig == 1)
name = 'Change first name in one of the questions'
desc = 'Take pairs that are originally predicted as duplicates, change first name in one of them and expect new prediction to be non-duplicate'
test = DIR(**t, expect=expect_fn, name=name, description=desc, capability='NER')
# test.run(new_pp)
# test.summary(3)
suite.add(test)


Changing first and last names

In [58]:
# t = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_names), nsamples=1500)
t = Perturb.perturb(parsed_qs, change_each_wrapper(Perturb.change_names), nsamples=1500)
name = 'Change first and last name in one of the questions'
desc = 'Take pairs that are originally predicted as duplicates, change first and last name in one of them and expect new prediction to be non-duplicate'
test = DIR(**t, expect=expect_fn, name=name, description=desc, capability='NER')
# test.run(new_pp)
# test.summary(3)
suite.add(test)


Locs

In [59]:
# t = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_location), nsamples=1500)
t = Perturb.perturb(parsed_qs, change_each_wrapper(Perturb.change_location), nsamples=1500)
name = 'Change location in one of the questions'
desc = 'Take pairs that are originally predicted as duplicates, change location in one of them and expect new prediction to be non-duplicate'
test = DIR(**t, expect=expect_fn, name=name, description=desc, capability='NER')
# test.run(new_pp)
# test.summary(3)
suite.add(test)

numbers

In [60]:
# t = Perturb.perturb(parsed_qs, wrap_apply_to_each(Perturb.change_number), nsamples=1500)
t = Perturb.perturb(parsed_qs, change_each_wrapper(Perturb.change_number), nsamples=1500)
name = 'Change numbers in one of the questions'
desc = 'Take pairs that are originally predicted as duplicates, change number in one of them and expect new prediction to be non-duplicate'
test = DIR(**t, expect=expect_fn, name=name, description=desc, capability='NER')
# test.run(new_pp)
# test.summary(3)
suite.add(test, overwrite=True)

In [61]:
### Change 

### Keep entities, fill in with BERT gibberish

In [62]:
def mask_gibberish(question):
    ents = question.ents
    if not ents:
        return None
    wp = [x.text for x in question if x.tag_ in ['WP', 'WRB', 'WDT']]
    if not wp:
        wp = question[0].text
    else:
        wp = wp[0]
    ents = [x.text for x in ents]
    ents[-1] = ents[-1] + '?'
    template = ' {mask} '.join([wp] + ents)
    gibberish = editor.template(template).data[:5]
#     return gibberish
    ret = [(question.text, x) for x in gibberish if question.text.lower() != x.lower() ]
    return ret
def gibberish_both(qs):
    q1, q2 = qs
    ret = []
    x1 = mask_gibberish(q1)
    if x1:
        ret.extend(x1)
    x2 = mask_gibberish(q2)
    if x2:
        ret.extend(x2)
    return ret
#     ret =  [(question.text, x[1]) for x in tg.fill_in_between([wp] + ents) if question.text != x[1]]
#     ret = [x for x in ret if len(x[1].split()) < len(x[0].split()) -4][:5]
#     return rett

In [63]:
t = Perturb.perturb(parsed_qs, gibberish_both, nsamples=500)
expect_false = Expect.eq(0)
name = 'Keep entitites, fill in with gibberish'
desc = 'Fill in between entitites with BERT, expect result to not be duplicate with original questions'
test = DIR(**t, expect=expect_false, name=name, description=desc, capability='NER')
# test.run(new_pp)
# test.summary(3)
suite.add(test, overwrite=True)

In [64]:
# test.summary(3)

## Temporal

Is != used to be

In [65]:
other_nouns = ['player', 'person', 'friend', 'kid', 'candidate']
nouns = list(set(professions + other_nouns))
t = editor.template(('Is {first_name} {last_name} {a:noun}?', 'Did {first_name} {last_name} use to be {a:noun}?'),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
name = 'Is person X != Did person use to be X'
test = MFT(**t, labels=0, name=name, description='', capability='Temporal')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

Is != becoming

In [66]:
t = editor.template(('Is {first_name} {last_name} {a:noun}?', 'Is {first_name} {last_name} becoming {a:noun}?'),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
name = 'Is person X != Is person becoming X'
test = MFT(**t, labels=0, name=name, description='', capability='Temporal')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

Before != after

In [67]:
t = editor.template((
    'What was {first_name} {last_name}\'s life before becoming {a:noun}?',
    'What was {first_name} {last_name}\'s life after becoming {a:noun}?'
),
                noun=nouns,
                adj=adjs,
                remove_duplicates=True, 
                nsamples=1000)
name = 'What was person\'s life before becoming X != What was person\'s life after becoming X'
test = MFT(**t, labels=0, name=name, description='', capability='Temporal')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [68]:
action12 = [x for x in editor.suggest('Do you have to {mask} your cat before {mask} it?') if 'kill' not in x[0] and 'kill' not in x[1]][:200]

In [69]:
t = editor.template((
    'Do you have to {a[0]} your {an} before {a[1]} it?',
    'Do you have to {a[0]} your {an} after {a[1]} it?'
),
    an = ['cat', 'dog', 'hamster'],
    a=action12,
    remove_duplicates=True, 
   nsamples=1000)
name = 'Do you have to X your dog before Y it != Do you have to X your dog after Y it.'
test = MFT(**t, labels=0, name=name, description='', capability='Temporal')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [70]:
print(', '.join(editor.suggest('Is is {mask} to eat after 10pm?')))

it, normal, OK, there, safe, best, ok, okay, acceptable, reasonable, permissible, legal, proper, wrong, healthy, good, enough, better, important, appropriate, left, allowed, possible, dinner, vegetarian, what, recommended, going, food, illegal, supposed, smart, unhealthy, hard, popular, hot, necessary, vegan, typical, not, free, right, required, fun, time, meant, something, one, kosher, customary, fair, mandatory, available, common, cheaper, alright, easier, suitable, feasible, polite, difficult, cool, certain, sensible, affordable, advisable, fine, fashionable, like, nice, unsafe, public, wise, decent, sufficient, harder, advised, traditional, usual, healthier, anyone, permitted, eating, easy, life, trendy, someone, forbidden, lawful, expensive, safer, new, everyone, compulsory, dangerous, worse, needed, realistic, temptation, essential, adequate, impossible, hungry, rude, risky, considered, prudent, taboo, different, tempting


In [71]:
mid = ['normal', 'ok', 'safe', 'dangerous', 'acceptable', 'reasonable', 'proper', 'wrong', 'healthy', 'important']

In [72]:
print(', '.join(editor.suggest('Is is {mid} to {mask} after 10pm?', mid=mid)))

sleep, drink, eat, work, leave, drive, go, smoke, stay, call, vote, talk, read, retire, visit, write, celebrate, post, do, party, watch, study, exercise, travel, fish, start, stop, walk, play, strike, disappear, be, tweet, live, continue, shop, rise, speak, relax, die, finish, blog, run, move, act, wait, text, cook, return, dance, bed, fly, quit, report, swim, pray, ask, rest, answer, close, gamble, enter, crash, think, marry, come, kill, look, meet, feed, operate, propose, vanish, dress, pee, linger, change, check, remain, dinner, protest, nap, withdraw, wake, fight, begin, publish, arrive, attend, skate, disturb, end, gather, complain, chat, exist, worry, know, cry, shower, cancel, see, open, perform, queue, respond, resign, exit, book, commute, function, avoid, film, buy, awake, binge, snack, clean, vape, park, happen, comment, blackout, occur, cycle, have, paint, vomit, panic, refuse, sit, say, resume, demonstrate, disconnect, kiss, feel, lose, search, hunt, bar, breathe, take, sur

In [73]:
activity = ['drink', 'sleep', 'drive', 'work', 'eat', 'smoke', 'walk', 'read', 'party', 'talk', 'exercise', 'celebrate', 'text', 'tweet', 'run', 'dance', 'swim', 'cook', 'pray', 'pee', 'rest']

In [74]:
t = editor.template(('Is it {mid} to {activity} before {hour}{ampm}?','Is it {mid} to {activity} after {hour}{ampm}?'),
                activity=activity,
                mid=mid,
                hour=[str(x) for x in range(1, 12)],
                ampm=['am', 'pm'],
                remove_duplicates=True, 
                nsamples=1000)
name = 'Is it {ok, dangerous, ...} to {smoke, rest, ...} after != before'
test = MFT(**t, labels=0, name=name, description='', capability='Temporal')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

## Negation

In [75]:
mid2 = mid + ['legal', 'awkward', 'socially acceptable']

In [76]:
print(', '.join(editor.suggest('Is it {mid} to {mask} in {country}?', mid=mid2)[:100]))

live, work, vote, drive, smoke, travel, be, marry, study, eat, fly, protest, pray, die, fight, speak, stay, shop, exist, campaign, gamble, meet, write, squat, visit, hunt, kill, murder, sleep, demonstrate, perform, drink, surf, serve, fish, gather, move, swim, remain, reside, pee, act, settle, cook, worship, dance, rape, play, strike, party, propose, fire, hide, report, train, march, burn, invest, paint, operate, race, shoot, barbecue, camp, preach, gay, arrive, hate, spy, vacation, migrate, sue, lie, talk, experiment, exercise, tan, search, interview, practice, volunteer, teach, survive, walk, land, buy, date, ski, ask, retire, wait, sit, holiday, communicate, breed, bomb, think, look, rent, diet


In [77]:
things = ['work', 'vote', 'travel', 'marry', 'drive', 'study', 'protest', 'campaign', 'fight', 'gamble', 'hunt', 'pray', 'smoke', 'fish', 'murder', 'invest', 'pee', 'march', 'worship', 'volunteer', 'surf', 'shoot', 'dance', 'camp', 'preach', 'spy', 'be gay', 'lie', 'divorce', 'discriminate']

In [78]:
tmp = editor.suggest(('How can I become a person who is {mask}', 'How can I become a person who is not {mask}?'))
tmp.remove('differently')
t = editor.template((
    'How can I become {a:x} person?',
    'How can I become a person who is not {x}?',
    ),
    x=tmp,
    remove_duplicates=True, 
    nsamples=1000)
name = 'How can I become a X person != How can I become a person who is not X' 
desc = ''
test = MFT(**t, labels=0, name=name, capability = 'Negation',
          description=desc)
# test.run(new_pp, n=500, seed=1)
# test.summary(n=5)
suite.add(test, overwrite=True)

In [79]:
t = editor.template(('Is it {mid} to {activity} in {country}?','Is it {mid} not to {activity} in {country}?'),
                activity=things,
                mid=mid2,
                remove_duplicates=True, 
                nsamples=1000)
name = 'Is it {ok, dangerous, ...} to {smoke, rest, ...} in country != Is it {ok, dangerous, ...} not to {smoke, rest, ...} in country'
test = MFT(**t, labels=0, name=name, description='', capability='Negation')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [80]:
t = editor.template((
    'What are things {a:noun} should worry about?',
    'What are things {a:noun} should not worry about?',
),
                noun=nouns,
                remove_duplicates=True, 
                nsamples=1000)
name = 'What are things a {noun} should worry about != should not worry about.'
test = MFT(**t, labels=0, name=name, description='', capability='Negation')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [81]:
# tmp

In [82]:
t = editor.template([(
    'How can I become {a:x[0]} person?',
    'How can I become a person who is not {x[1]}?',
    ),
    (
    'How can I become {a:x[1]} person?',
    'How can I become a person who is not {x[0]}?',
    ),
],
    unroll=True,
    x=antonyms,
    remove_duplicates=True, 
    nsamples=1000)
name = 'How can I become a X person == How can I become a person who is not antonym(X)' 
desc = ''
test = MFT(**t, labels=1, name=name, capability = 'Negation',
          description=desc)
# test.run(new_pp, n=500, seed=1)
# test.summary(n=3)
suite.add(test, overwrite=True)

## Coref

In [83]:
t = editor.template(
    [(
        'If {male} and {female} were alone, do you think he would reject her?',
        'If {male} and {female} were alone, do you think she would reject him?',
    ),
        (
        'If {female} and {male} were alone, do you think he would reject her?',
        'If {female} and {male} were alone, do you think she would reject him?',
    )
    ],
    remove_duplicates=True, 
    nsamples=1000,
    unroll=True)
name = 'Simple coref: he and she'
desc = '' 
test = MFT(**t, labels=0, name=name, description=desc, capability='Coref')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [84]:
t = editor.template(
    [(
        'If {male} and {female} were married, would his family be happy?',
        'If {male} and {female} were married, would {female}\'s family be happy?',
    ),(
        'If {male} and {female} were married, would her family be happy?',
        'If {male} and {female} were married, would {male}\'s family be happy?',
    ),
    ]
        ,
    unroll=True,
    remove_duplicates=True, 
    nsamples=1000)
name = 'Simple coref: his and her'
desc = '' 
test = MFT(**t, labels=0, name=name, description=desc, capability='Coref')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test, 'Simple coref: his and her', 'Coref', 'TODO_DESCRIPTION')

## SRL

In [85]:
print(', '.join(editor.suggest('Who is the best {mask} in the world?')))

chef, pizza, boxer, player, footballer, athlete, rapper, actor, singer, cook, magician, robot, coach, beer, cyclist, wrestler, drummer, musician, quarterback, hacker, dog, baker, fighter, journalist, restaurant, steak, teacher, doctor, gamer, band, shooter, DJ, football, person, horse, photographer, driver, fisherman, burger, coffee, lawyer, writer, food, dancer, student, wine, artist, man, surgeon, comedian, trainer, VPN, vegetarian, programmer, team, game, vegan, guitarist, goalkeeper, server, guy, cricket, sport, engineer, dentist, waiter, cheese, bartender, fish, chicken, goalie, car, manager, computer, referee, hunter, guitar, basketball, tennis, astronaut, mathematician, sandwich, blogger, accountant, TV, soldier, sniper, job, runner, barbecue, chocolate, scientist, friend, cat, gun, pitcher, judge, AI, philosopher, butcher, painter, golf, camel, translator, piano, worker, farmer, conductor, politician, bike


In [86]:
thing = ['chef', 'boxer', 'player', 'footballer', 'athlete', 'rapper', 'actor', 'singer', 'cook', 'magician', 'coach', 'cyclist', 'wrestler', 'drummer', 'musician', 'quarterback', 'hacker', 'baker', 'fighter', 'journalist', 'teacher', 'doctor', 'gamer', 'husband', 'DJ', 'person', 'man', 'woman', 'surgeon', 'comedian', 'trainer', 'programmer', 'guitarist', 'goalkeeper']

In [87]:
print(', '.join(editor.suggest('Who do {mask} think is the the best {thing} in the world?', thing=thing)))

you, YOU, people, we, they, I, readers, You, guys, u, your, fans, ya, others, experts, Americans, scientists, men, voters, some, the, students, analysts, authors, everyone, friends, conservatives, players, critics, Canadians, he, judges, folks, women, all, most, historians, i, coaches, supporters, members, journalists, researchers, celebrities, many, viewers, liberals, participants, audiences, Australians, respondents, editors, Republicans, artists, writers, U, pundits, yo, comedians, gamers, reporters, economists, teachers, parents, veterans, commentators, consumers, anyone, users, independents, kids, pros, millennials, each, individuals, YOUR, those, athletes, doctors, candidates, reviewers, ye, investigators, both, politicians, scholars, not, two, philosophers, atheists, agents, humans, stars, ladies, police, feminists, insiders, yours, any, archaeologists, leaders, Christians, seniors, Democrats, contestants, psychologists, adults, guests, teams, astronomers, competitors, professio

In [88]:
subjects = ['you', 'people', 'readers', 'guys', 'fans', 'experts', 'scientists', 'Americans', 'students', 'men', 'voters', 'authors', 'conservatives', 'women', 'Canadians', 'analysts', 'critics', 'judges', 'artists', 'researchers', 'liberals', 'historians', 'Australians', 'journalists', 'Republicans', 'coaches', 'parents', 'kids', 'economists', 'reporters', 'consumers', 'veterans', 'doctors']

In [89]:
print(', '.join(editor.suggest('Who do {subjects} think is the the {mask} {thing} in the world?', thing=thing, subjects=subjects)[:50]))

best, greatest, worst, top, smartest, finest, strongest, fastest, biggest, toughest, deadliest, coolest, hottest, better, happiest, safest, hardest, great, oldest, elite, richest, leading, premier, brightest, busiest, ultimate, superior, youngest, largest, favorite, Greatest, BEST, highest, professional, easiest, most, foremost, newest, Best, perfect, outstanding, dominant, premiere, first, star, next, only, weakest, wealthiest, quickest


In [90]:
best = ['best', 'greatest', 'worst', 'top', 'smartest', 'strongest', 'finests', 'happiest', 'coolest', 'richest', 'leading', 'brightest', 'premier', 'ultimate', 'dominant']

In [91]:
t = editor.template((
    'Who do {subjects} think is the {best} {thing} in the world?',
    'Who is the {best} {thing} in the world according to {subjects}?'
),
    subjects=subjects,
    best=best,
    thing=thing,
    remove_duplicates=True, 
    nsamples=1000)
name = 'Who do X think - Who is the ... according to X'
desc = '' 
test = MFT(**t, labels=1, name=name, description=desc, capability='SRL')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [92]:
print(', '.join([str(x) for x in editor.suggest('Are {mask} smaller than {a}?', a=['bananas', 'dogs', 'cars', 'cats', 'elephants'])][:100]))

humans, cats, mice, dogs, pigs, people, you, birds, chickens, cows, sheep, rats, fish, bears, rabbits, monkeys, elephants, we, snakes, bees, bats, lions, children, puppies, spiders, babies, ants, insects, they, kittens, dolphins, butterflies, frogs, animals, robots, turtles, ducks, trees, bugs, kids, flies, worms, whales, mosquitoes, crabs, cars, plants, sharks, dinosaurs, beetles, horses, tigers, wolves, primates, cattle, chimpanzees, goats, apes, men, deer, reptiles, rodents, mammals, balls, pets, ponies, carrots, seals, potatoes, apples, mushrooms, dragons, ticks, boys, toys, girls, us, computers, things, stones, women, brains, eggs, bulls, calves, Indians, dwarves, bananas, houses, burgers, flowers, shells, beans, diamonds, coins, trolls, guys, drones, bacteria, Lions


In [93]:
things = editor.suggest('Are {mask} smaller than {a}?',a=['bananas', 'dogs', 'cars', 'cats', 'elephants'] )[:100]

In [94]:
print(', '.join([str(x) for x in editor.suggest('Are {a} {mask} than {a2}?', a=things)][:100]))

smarter, better, bigger, faster, stronger, cooler, different, smaller, worse, larger, tougher, more, taller, safer, slower, wiser, heavier, weaker, healthier, happier, lighter, nicer, less, older, greater, quicker, cheaper, harder, shorter, cleaner, longer, quieter, brighter, easier, darker, hotter, higher, closer, other, colder, thicker, younger, louder, warmer, lower, sharper, deeper, wider, lesser, softer, broader, simpler, thinner, intelligent, clearer, stranger, important, dangerous, farther, rather, fewer, richer, smoother, finer, differently, smart, tighter, poorer, earlier, sooner, superior, real, related, stupid, wealthier, anymore, inferior, normal, valuable, dumb, interesting, beautiful, weird, narrower, big, further, stricter, strange, human, safe, powerful, rare, MORE, fuller, dead, special, alive, similar, old, newer


In [95]:
comp = ['better', 'worse', 'cheaper', 'bigger', 'louder', 'longer', 'larger', 'smaller', 'warmer', 'colder', 'thicker', 'lighter', 'heavier']

Order doesn't matter for comparison

In [96]:
t = editor.template([
    (
    'Are {t1} {comp} than {t2}?',
    'What is {comp}, {t2} or {t1}?'
    ),
    (
    'Are {t1} {comp} than {t2}?',
    'Are {t2} {comp} than {t1}?',
    ),
    (
    'Are {t1} {comp} than {t2}?',
    'What is {comp}, {t1} or {t2}?',
    )
]
    ,
    t = things,
    comp = comp,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'Order does not matter for comparison'
desc = '' 
test = MFT(**t, labels=1, name=name, description=desc, capability='SRL')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [97]:
# editor.template('Is {first_name1} {mask} {first_name2}?', remove_duplicates=True)[:100]

In [98]:
print(', '.join(editor.suggest('Is {first_name1} {mask} to {first_name2}?', remove_duplicates=True)[:100]))
print()
print(', '.join(editor.suggest('Is {first_name1} {mask} {first_name2}?', remove_duplicates=True)[:100]))

related, married, close, engaged, talking, lying, connected, speaking, closer, loyal, linked, attracted, going, referring, listening, important, faithful, similar, mean, responding, writing, hostile, true, dead, tied, next, bound, known, indebted, lied, proposing, closest, opposed, truthful, committed, right, up, turning, kind, Married, kin, proposed, attached, abusive, gay, good, real, returning, allergic, straight, available, relevant, talk, attractive, fair, written, nice, dangerous, crazy, new, alive, coming, on, friendly, devoted, cruel, entitled, special, superior, answer, addicted, reacting, out, back, born, supposed, equal, father, drawn, happy, happening, answering, acceptable, nicer, spoken, getting, open, decent, familiar, heir, guilty, talked, mentioned, gone, sympathetic, helpful, insane, down, pregnant, equivalent

or, really, and, with, actually, not, an, Really, still, like, /, &, born, a, from, called, real, indeed, now, the, dating, truly, after, even, married, to, ma

In [99]:
symmetric = ['dating', 'married to', 'close to', 'engaged to', 'connected to', 'married to', 'friends with', 'related to', 'an acquaintance of']

In [100]:
t = editor.template((
    'Is {first_name1} {s} {first_name2}?',
#     'Are {t2} {comp} than {t1}?',
#     'What is {comp}, {t1} or {t2}?',
    'Is {first_name2} {s} {first_name1}?',
),
    s = symmetric,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'Order does not matter for symmetric relations'
desc = 'e.g. dating, married to, close to, engaged to, etc' 
test = MFT(**t, labels=1, name=name, description=desc, capability='SRL')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

Order matters for asymetric relations

In [101]:
asymmetric = ['hurting', 'lying to', 'loyal to', 'faithful to', 'proposing to', 'indebted to', 'abusive to', 'using', 'expecting', 'beating', 'punching', 'raising', 'poisoning', 'protecting', 'kidnapping']

In [102]:
t = editor.template((
    'Is {first_name1} {s} {first_name2}?',
#     'Are {t2} {comp} than {t1}?',
#     'What is {comp}, {t1} or {t2}?',
    'Is {first_name2} {s} {first_name1}?',
),
    s = asymmetric,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'Order does matter for asymmetric relations'
desc = 'e.g. hurting lying to, faithful to, etc'
test = MFT(**t, labels=1, name=name, description=desc, capability='SRL')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

More traditional SRL

In [103]:
print(', '.join(editor.suggest('Did John buy the {mask}?', remove_duplicates=True)[:100]))

farm, stake, house, property, company, land, rights, ticket, papers, newspaper, book, island, estate, ranch, boat, horse, paper, shares, tickets, business, gun, books, newspapers, yacht, team, ship, stock, idea, plot, contract, Bible, phone, franchise, shotgun, store, horses, cattle, church, place, game, factory, castle, club, painting, bank, tract, rifle, manuscript, car, campaign, Ark, slaves, plane, school, money, guns, sword, plantation, beer, trust, election, film, ring, building, station, twins, deal, loan, time, tractor, home, letter, IRA, plan, title, watch, story, letters, rest, cows, piece, diary, brewery, charter, diamond, mortgage, hospital, site, post, operation, insurance, railroad, cow, coins, Dell, case, children, wine, telephone, dog


In [104]:
obj = ['farm', 'house', 'property', 'company', 'land', 'ticket', 'newspaper', 'book', 'island', 'estate', 'ranch', 'boat', 'horse', 'paper', 'business', 'gun', 'game', 'factory', 'castle', 'painting', 'rifle', 'car', 'school', 'building']

In [105]:
print(', '.join(editor.suggest('Did John {mask} the {obj}?', obj=obj, remove_duplicates=True)[:100]))

buy, get, take, sell, have, lose, own, leave, see, keep, want, use, win, need, steal, manage, find, miss, receive, handle, inherit, purchase, abandon, seize, control, run, return, remember, finish, break, enjoy, know, hold, give, drop, claim, recover, move, touch, save, land, pull, on, destroy, share, start, rebuild, crash, call, stop, afford, hit, acquire, join, like, bought, deliver, lead, remove, clear, fix, sign, split, quit, watch, bring, play, survive, left, carry, sold, shoot, maintain, close, grab, settle, reclaim, accept, retain, enter, complete, change, possess, raise, make, wreck, beat, flip, catch, nail, build, forget, ruin, recall, try, retrieve, surrender, notice, mention, got


In [106]:
import pattern
import pattern.en
verbs = ['buy', 'purchase', 'sell', 'leave', 'own', 'take', 'keep', 'want', 'lose', 'destroy', 'inherit', 'find', 'use', 'need', 'receive', 'return', 'like', 'enjoy', 'abandon', 'manage', 'remember', 'miss', 'move', 'seize', 'steal']
a = pattern.en.tenses('stolen')[0]
verbs = [(v, pattern.en.conjugate(v, *a)) for v in verbs]
verbs[3] = ('leave', 'left')
verbs

[('buy', 'bought'),
 ('purchase', 'purchased'),
 ('sell', 'sold'),
 ('leave', 'left'),
 ('own', 'owned'),
 ('take', 'taken'),
 ('keep', 'kept'),
 ('want', 'wanted'),
 ('lose', 'lost'),
 ('destroy', 'destroyed'),
 ('inherit', 'inherited'),
 ('find', 'found'),
 ('use', 'used'),
 ('need', 'needed'),
 ('receive', 'received'),
 ('return', 'returned'),
 ('like', 'liked'),
 ('enjoy', 'enjoyed'),
 ('abandon', 'abandoned'),
 ('manage', 'managed'),
 ('remember', 'remembered'),
 ('miss', 'missed'),
 ('move', 'moved'),
 ('seize', 'seized'),
 ('steal', 'stolen')]

In [107]:
t = editor.template((
    'Did {first_name} {verb[0]} the {obj}?',
    'Was the {obj} {verb[1]} by {first_name}?'
),
    verb=verbs,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'traditional SRL: active / passive swap'
desc = ''
test = MFT(**t, labels=1, name=name, description=desc, capability='SRL')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [108]:
t = editor.template((
    'Did {first_name} {verb[0]} the {obj}?',
    'Was {first_name} {verb[1]} by the {obj}?'
),
    verb=verbs,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'traditional SRL: wrong active / passive swap'
desc = ''
test = MFT(**t, labels=0, name=name, description=desc, capability='SRL')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

With people

In [109]:
print(', '.join(editor.suggest('Does {first_name} {mask} {first_name2}?', remove_duplicates=True)[:100]))

know, marry, kill, love, hate, remember, like, want, and, or, have, date, need, see, meet, tell, Know, find, recognize, choose, understand, forgive, trust, mention, blame, follow, murder, miss, bother, ask, deserve, save, beat, mean, attack, prefer, dislike, resemble, become, survive, support, get, help, visit, kiss, leave, accept, recognise, married, own, notice, be, hurt, call, replace, hit, take, fear, catch, reject, clone, divorce, believe, kidnap, Love, /, Remember, adopt, succeed, question, influence, killed, recall, shoot, contact, raise, respect, rape, use, judge, include, eat, v, owe, confront, join, KILL, despise, Like, sponsor, hire, crush, control, threaten, stalk, destroy, Want, &, defeat, lose


In [110]:
pverb = ['love', 'hate', 'like', 'remember', 'recognize', 'trust', 'deserve', 'understand', 'blame', 'dislike', 'prefer', 'follow', 'notice', 'hurt', 'bother', 'support', 'believe', 'accept', 'attack']
a = pattern.en.tenses('stolen')[0]
pverb = [(v, pattern.en.conjugate(v, *a)) for v in pverb]
t = editor.template((
    'Does {first_name} {verb[0]} {first_name2}?',
    'Is {first_name2} {verb[1]} by {first_name}?',
),
    verb=pverb,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'traditional SRL: active / passive swap with people'
desc = ''
test = MFT(**t, labels=1, name=name, description=desc, capability='SRL')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [111]:
pverb = ['love', 'hate', 'like', 'remember', 'recognize', 'trust', 'deserve', 'understand', 'blame', 'dislike', 'prefer', 'follow', 'notice', 'hurt', 'bother', 'support', 'believe', 'accept', 'attack']
a = pattern.en.tenses('stolen')[0]
pverb = [(v, pattern.en.conjugate(v, *a)) for v in pverb]
t = editor.template((
    'Does {first_name} {verb[0]} {first_name2}?',
    'Is {first_name} {verb[1]} by {first_name2}?',
),
    verb=pverb,
    obj=obj,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'traditional SRL: wrong active / passive swap with people'
desc = ''
test = MFT(**t, labels=0, name=name, description=desc, capability='SRL')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

## Logic

In [112]:
t = editor.template((
    'Is {first_name} {last_name} {a:p1} ?',
    'Is {first_name} {last_name} {a:p3}?',
),
    p=professions,
    remove_duplicates=True, 
    nsamples=1000)
test = MFT(**t, labels=0)
# test.run(new_pp)
# test.summary(n=3)


In [113]:
t = editor.template((
    'Is {first_name} {last_name} {a:p1} or {a:p2}?',
    'Is {first_name} {last_name} simultaneously {a:p3} and {a:p4}?',
),
    p=professions,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'A or B is not the same as C and D'
desc = ''
test = MFT(**t, labels=0, name=name, description=desc, capability='Logic')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)


In [114]:
t = editor.template((
    'Is {first_name} {last_name} {a:p1} or {a:p2}?',
    'Is {first_name} {last_name} simultaneously {a:p1} and {a:p2}?',
),
    p=professions,
    remove_duplicates=True, 
    nsamples=1000)
name = 'A or B is not the same as A and B'
desc = ''
test = MFT(**t, labels=0, name=name, description=desc, capability='Logic')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [115]:

t = editor.template((
    'Is {first_name} {last_name} {a:p1} {andor} {a:p2}?',
    'Is {first_name} {last_name} {a:p2} {andor} {a:p1}?',
),
    andor=['and', 'or'],
    p=professions,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'A and / or B is the same as B and / or A'
desc = ''
test = MFT(**t, labels=1, name=name, description=desc, capability='Logic')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [116]:

t = editor.template((
    'Is {first_name} {last_name} {a:nat} {p1}?',
    'Is {first_name} {last_name} {a:p1} and {nat}?',
),
    nat=editor.lexicons['nationality'][:20],
    p=professions,
    remove_duplicates=True, 
    nsamples=1000)
# data = [tuple(np.random.choice(x, 2, replace=False)) for x in data]
name = 'a {nationality} {profession} = a {profession} and {nationality}'
desc = ''
test = MFT(**t, labels=1, name=name, description=desc, capability='Logic')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

Reflexivity

In [117]:
t = Perturb.perturb(list(all_questions), lambda x:(x, x), nsamples=1000, keep_original=False)
name = 'Reflexivity: (q, q) should be duplicate'
desc = ''
test = MFT(**t, labels=1, name=name, description=desc, capability='Logic')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)
# test.summary(n=3)

Symmetry

In [118]:
t = Perturb.perturb(qs, lambda x:(x[1], x[0]), nsamples=500, keep_original=True)
name = 'Symmetry: f(a, b) = f(b, a)'
desc = ''
test = INV(t.data, name=name, description=desc, capability='Logic')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [119]:
# t.data[23]

In [120]:
# labels

In [121]:
import collections
def extract_unknown_implications(pairs, labels):
    graph = collections.defaultdict(lambda: set())
    ls = {}
    for x, y in zip(pairs, labels):
        graph[x[0]].add(x[1])
        graph[x[1]].add(x[0])
        t = tuple(sorted(x))
        ls[t] = y

    d = []
    l = []
    for x in graph:
        if len(graph[x]) == 1:
            continue
        for y in graph[x]:
            t = tuple(sorted((x, y)))
    #         print(t, ls[t])
        new = list(set([tuple(sorted(a)) for a in itertools.product(list(graph[x]), list(graph[x])) if a[0] != a[1]]))
        new = [a for a in new if a not in ls]
        for b, c in new:
            t1 = tuple(sorted((x, b)))
            t2 = tuple(sorted((x, c)))
            l1 = ls[t1]
            l2 = ls[t2]
            if l1 + l2 == 2:
                l3 = 1
            elif l1 + l2 == 1:
                l3 = 0
            else:
                continue
            new_x = [(x, b), (x, c), (b, c)]
            new_l = np.array([l1, l2, l3])
            d.append(new_x)
            l.append(new_l)
    return d, l

In [122]:
data, ls = extract_unknown_implications(qs, labels)

In [123]:
def expect_triplet(xs, preds, confs, labels, meta=None):
    if (preds[0] + preds[1]) == 2:
        if preds[2] != 1:
            return np.array([-3, -2, -1])
        else:
            return np.array([True, True, True])
    if (preds[0] + preds[1] == 1) and preds[1] != 0:
        if preds[1] != 0:
            return np.array([-3, -2, -1])
        else:
            return np.array([True, True, True])
    return None
#     if preds[0] != labels[0] or preds[1] != labels[1]:
#         return None
#     if preds[2] == labels[2]:
#         return np.array([True, True, True])
#     else:
#         return np.array([-3, -2, -1])
expect = Expect.testcase(expect_triplet)

In [124]:
name = 'Testing implications'
desc = 'f(x, a) = 1 and f(x, b) = 1 => f(a, b) = 1\nf(x, a) = 1 and f(x, b) = 0 => f(a, b) = 0\n Only used (x, a, b) such that (x, a) and (x, b) in val dataset and (a, b) is not.\n Expectation function filters out examples where f(x, a) or f(x, b) are incorrect'
test = DIR(data, expect, labels=ls, name=name, description=desc, capability='Logic')
# test.run(new_pp)
# test.summary(n=3)
suite.add(test)

In [126]:
suite.save('/home/marcotcr/tmp/qqp_suite.pkl')

## Bert

In [129]:
suite.run(new_pp, n=500, seed=1)

Running Modifier: adj
Predicting 500 examples
Running different adjectives
Predicting 500 examples
Running Different animals
Predicting 500 examples
Running Irrelevant modifiers - animals
Predicting 500 examples
Running Irrelevant modifiers - people
Predicting 500 examples
Running Irrelevant preamble with different examples.
Predicting 500 examples
Running Preamble is relevant (different injuries)
Predicting 500 examples
Running How can I become more {synonym}?
Predicting 500 examples
Running (question, f(question)) where f(question) replaces synonyms?
Predicting 326 examples
Running Replace synonyms in real pairs
Predicting 696 examples
Running How can I become more X != How can I become less X
Predicting 500 examples
Running How can I become more X = How can I become less antonym(X)
Predicting 500 examples
Running add one typo
Predicting 1500 examples
Running contrations
Predicting 1468 examples
Running (q, paraphrase(q))
Predicting 18812 examples
Running Product of paraphrases(q1) *

In [130]:
suite.save('/tmp/bert')

In [131]:
suite.summary(n=3)

Vocabulary

Modifier: adj
Test cases:      1000
Test cases run:  500
Fails (rate):    374 (74.8%)

Example fails:
1.0 ('Is Anthony Peterson an investor?', 'Is Anthony Peterson an active investor?')
----
0.9 ('Is Jessica Brooks an engineer?', 'Is Jessica Brooks an American engineer?')
----
1.0 ('Is Andrew Hughes a nurse?', 'Is Andrew Hughes an excellent nurse?')
----


different adjectives
Test cases:      954
Test cases run:  500
Fails (rate):    1 (0.2%)

Example fails:
0.7 ('Is Ashley Ramirez Indian?', 'Is Ashley Ramirez American?')
----


Different animals
Test cases:      928
Test cases run:  500
Fails (rate):    28 (5.6%)

Example fails:
0.9 ('Can I feed my rat poop?', 'Can I feed my cat poop?')
----
0.8 ('Can I feed my lobster formula?', 'Can I feed my spider formula?')
----
0.7 ('Can I feed my lizard corn?', 'Can I feed my fish corn?')
----


Irrelevant modifiers - animals
Test cases:      1000
Test cases run:  500
Fails (rate):    0 (0.0%)


Irrelevant modifiers - people
Test c

## Roberta

In [132]:
suite = TestSuite.from_file('/home/marcotcr/tmp/qqp_suite.pkl')

In [133]:
suite.run(new_pp, n=500, seed=1)

Running Modifier: adj
Predicting 500 examples
Running different adjectives
Predicting 500 examples
Running Different animals
Predicting 500 examples
Running Irrelevant modifiers - animals
Predicting 500 examples
Running Irrelevant modifiers - people
Predicting 500 examples
Running Irrelevant preamble with different examples.
Predicting 500 examples
Running Preamble is relevant (different injuries)
Predicting 500 examples
Running How can I become more {synonym}?
Predicting 500 examples
Running (question, f(question)) where f(question) replaces synonyms?
Predicting 326 examples
Running Replace synonyms in real pairs
Predicting 696 examples
Running How can I become more X != How can I become less X
Predicting 500 examples
Running How can I become more X = How can I become less antonym(X)
Predicting 500 examples
Running add one typo
Predicting 1500 examples
Running contrations
Predicting 1468 examples
Running (q, paraphrase(q))
Predicting 18812 examples
Running Product of paraphrases(q1) *

In [134]:
suite.save('/tmp/roberta')

In [135]:
suite.summary(n=3)

Vocabulary

Modifier: adj
Test cases:      1000
Test cases run:  500
Fails (rate):    372 (74.4%)

Example fails:
0.8 ('Is Zachary Brown an intern?', 'Is Zachary Brown a bad intern?')
----
1.0 ('Is Elizabeth Stewart an engineer?', 'Is Elizabeth Stewart an elite engineer?')
----
0.8 ('Is Jeffrey Lopez a producer?', 'Is Jeffrey Lopez an experienced producer?')
----


different adjectives
Test cases:      954
Test cases run:  500
Fails (rate):    1 (0.2%)

Example fails:
0.8 ('Is Kyle Cruz Indian?', 'Is Kyle Cruz an immigrant?')
----


Different animals
Test cases:      928
Test cases run:  500
Fails (rate):    0 (0.0%)


Irrelevant modifiers - animals
Test cases:      1000
Test cases run:  500
Fails (rate):    0 (0.0%)


Irrelevant modifiers - people
Test cases:      987
Test cases run:  500
Fails (rate):    0 (0.0%)


Irrelevant preamble with different examples.
Test cases:      938
Test cases run:  500
Fails (rate):    497 (99.4%)

Example fails:
0.0 ('My pet turtle eats soy. Is it nor

In [136]:
def fail_rate(self):
    n_run = n = len(self.data)
    if self.run_idxs is not None:
        n_run = len(self.run_idxs)
    fails = self.fail_idxs().shape[0]
    filtered = self.filtered_idxs().shape[0]
    nonfiltered = n_run - filtered
    if nonfiltered == 0:
        return 0
    return 100 * fails / nonfiltered

fail = collections.defaultdict(lambda: collections.defaultdict(lambda: 0))
names = ['bert', 'roberta']
for name in names:
    suite = TestSuite.from_file('/tmp/%s' % name)
    for t in suite.tests:
        fail[t][name] = fail_rate(suite.tests[t])

In [140]:
suite = TestSuite.from_file('/tmp/bert')

In [266]:
t = 'How can I become a X person == How can I become a person who is not antonym(X)'
suite.tests[t].summary(3)

Test cases:      2000
Test cases run:  500
Fails (rate):    386 (77.2%)

Example fails:
0.1 ('How can I become a positive person?', 'How can I become a person who is not negative?')
----
0.2 ('How can I become a good person?', 'How can I become a person who is not bad?')
----
0.1 ('How can I become a conservative person?', 'How can I become a person who is not progressive?')
----


In [141]:
for t in suite.tests:
    print(t)
    print('%.1f & %.1f' % (tuple([fail[t][n] for n in names])))
    suite.tests[t].summary(3)
    print()

Modifier: adj
74.8 & 74.4
Test cases:      1000
Test cases run:  500
Fails (rate):    374 (74.8%)

Example fails:
1.0 ('Is Alexander Williams an accountant?', 'Is Alexander Williams an elite accountant?')
----
0.9 ('Is Kelly Bennett an editor?', 'Is Kelly Bennett a successful editor?')
----
0.8 ('Is Noah Gonzalez a journalist?', 'Is Noah Gonzalez an English journalist?')
----

different adjectives
0.2 & 0.2
Test cases:      954
Test cases run:  500
Fails (rate):    1 (0.2%)

Example fails:
0.7 ('Is Ashley Ramirez Indian?', 'Is Ashley Ramirez American?')
----

Different animals
5.6 & 0.0
Test cases:      928
Test cases run:  500
Fails (rate):    28 (5.6%)

Example fails:
0.8 ('Can I feed my turtle insulin?', 'Can I feed my snake insulin?')
----
0.7 ('Can I feed my lizard corn?', 'Can I feed my fish corn?')
----
0.8 ('Can I feed my snail salad?', 'Can I feed my turtle salad?')
----

Irrelevant modifiers - animals
0.0 & 0.0
Test cases:      1000
Test cases run:  500
Fails (rate):    0 (0.